In [4]:
import bcrypt
import hashlib
import os

def PassStrengthChecker(password):
    score =0
    feedback = []
    common_passwords = [
        "password", "123456", "111111",
    ]
    if password.lower() in common_passwords:
        feedback.append("password is common")
        return {"score":0, "feedback":feedback, "entropy": 0,"rating":"very weak"}
    
    length =len(password)
    if length >=8:
        score +=1
    if length >= 12:
        score+= 1

    hasLower= False
    hasUpper= False
    hasDigit= False
    hasSymbol= False
    
    def log2(x):
        result = 0
        while x > 1:
            x /= 2.0
            result += 1
        return result


    for c in password:
        if 'a' <=c<= 'z':
            hasLower =True
        elif 'A' <=c<='Z':
            hasUpper=True
        elif '0'<=c<='9':
            hasDigit =True
        else:
            hasSymbol =True

    pool = 0
    if hasLower:
        pool +=26
        score +=1
    if hasUpper:
        pool +=26
        score +=1
    if hasDigit:
        pool +=10
        score +=1
    if hasSymbol:
        pool +=32  
        score +=1

    entropy = 0
    
    if pool > 0:
        entropy = round(length * log2(pool), 2)

    if score <=2:
        rating = "weak"
    elif score <=4:
        rating = "medium"
    elif score<= 6:
        rating= "strong"
    else:
        rating= "very strong"

    if length< 8:
        feedback.append("too short make at least 8 characters long")
    elif length <12:
        feedback.append("could be better")
    else:
        feedback.append("good length")

    if not hasLower:
        feedback.append("add lowercase")
    if not hasUpper:
        feedback.append("add uppercase")
    if not hasSymbol:
        feedback.append("add special characters")
    if not hasDigit:
        feedback.append("add numbers")


    return {
        "score": score,
        "rating": rating,
        "feedback": feedback,
        "entropy": entropy
    }


if __name__ == "__main__":
    Entry = input("Enter pass: ")
    result = PassStrengthChecker(Entry)
    print(f"Score: {result['score']}out of 7")
    print(f"Entropy: {result['entropy']}")
    print(f"Rating: {result['rating']}")
    for msg in result["feedback"]:
        print(msg)


Score: 4out of 7
Entropy: 72
Rating: medium
good length
add uppercase
add special characters


In [2]:
def hashSha(password, salt = None):
    if salt is None:
        salt = os.urandom(16)
    y = salt + password.encode("utf-8")
    d = hashlib.sha256(y).hexdigest()
    return y, salt
def hashMd5(password):
    bytess = password.encode("utf-8")
    x = hashlib.md5(bytess).hexdigest()
    return x
def hashBcrypt(password, rounds):
    rounds = 12
    bytess = password.encode("utf-8")
    salt = bcrypt.gensalt(rounds)
    hashed = bcrypt.hashpw(bytess, salt)
    return hashed
def verify_bcrypt(password, storedhash):
    if isinstance(storedhash, str):
        storedhash = storedhash.encode("utf-8")
    bytess = password.encode("utf-8")
    return bcrypt.checkpw(bytess, storedhash)

if __name__ =="__main__":
    Entry = input("Enter a pass:")
    md5_digest = hashMd5(Entry)
    print("MD5:", md5_digest)
    sha_digest, sha_salt = hashSha(Entry)
    print("SHA256:", sha_digest)
    b_hash = hashBcrypt(Entry, rounds=12)
    print("bcrypt hash:",b_hash.decode("utf-8"))

MD5: 3791082c42f527275fbfd02cf3ea5244
SHA256: b'\x93\xdb\x9d\x8e80z\xd9\xf4u+\xebG\x14\xa2Miamtheone99'
bcrypt hash: $2b$12$OTtA2a3yCnPNr16dfJH7feEssKqQTlWgxVmrpAyVWmrq3v77rzFwq


In [3]:
def shaHex(data):
    return hashlib.sha256(data).hexdigest()
def noSalt(password):
    bytess =password.encode("utf-8")
    return shaHex(bytess)
def salt(password, salt=None):
    if salt is None:
        salt =os.urandom(16)
    bytess =password.encode("utf-8")
    return salt, shaHex(salt + bytess)
def Pepper(password, pepper, salt=None):
    if isinstance(pepper, str):
        pepper =pepper.encode("utf-8")
    if salt is None:
        salt =os.urandom(16)
    bytess= password.encode("utf-8")
    return salt, shaHex(salt + bytess + pepper)

pas = "user123password"
pepper = "pepper"

nosalt = noSalt(pas)
print("no salt", nosalt)
salt, salted = salt(pas)
print("salt", f"salt={salt.hex()}, hash={salted}")
salt2, pepperr = Pepper(pas, pepper)
print("pepper", f"salt={salt2.hex()}, hash={pepperr}")

no salt 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
salt salt=e293c4fe8ee23411eb68f998bd0b3320, hash=2205ecca0784220312fd202d7dcc88b88a335ce193b07cec6d020075e513d22d
pepper salt=59305b74b4ef2748e7a2ac361c2942cf, hash=35be15a88bc2026e916078273dafb5e5a845b3e6ac82adb1d01c5ae754041da1


In [ ]:
import pyotp
import qrcode
import time

RanValue = pyotp.random_base32()

issuer = "lab3"
accountName = "test@test.com"

link = pyotp.totp.TOTP(RanValue).provisioning_uri(
    name=accountName,
    issuerName= issuer
)
print(link)
img =qrcode.make(link)
img.save("QrCode.png")
totp = pyotp.TOTP(RanValue)

print("\nTOTP code:")
print(totp.now())



otpauth://totp/ExampleApp:user%40example.com?secret=3ZDJ6AQMXV62YHXQRYRP4CVXSOYLFFPQ&issuer=ExampleApp

TOTP code:
202571


In [ ]:

def hashMd5(password):
    return hashlib.md5(password.encode("utf-8")).hexdigest()

def noSalt(password):
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

data = [
    "password",
    "123456",
    "11111",
    "username"
]

def CrackHash(target, type):
    start = time.time()
    for i in data:
        if type=="md5":
            attempt=hashMd5(i)
        elif type=="sha256":
            attempt=noSalt(i)
        else:
            return

        if attempt == target:
            end = time.time()
            print(f"Match found. Password is {i}")
            print(f"Time taken: {end-start} seconds")
            return

    print("no match")

md5 = hashMd5("password")
sha256 =noSalt("password")

print("\n MD5 Demonstration ")
CrackHash(md5, "md5")
print("\n-SHA256 Demonstration ")
CrackHash(sha256, "sha256")




 MD5 Demonstration 
Match found. Password is password
Time taken: 6.4373016357421875e-06 seconds

-SHA256 Demonstration 
Match found. Password is password
Time taken: 3.5762786865234375e-06 seconds


In [ ]:
# REFLECTION 
# During this seminar, I have gained an understanding of the application of hashing and protection techniques to secure passwords. I found out the importance of incorporating different layers like salts and peppers and how implementing MD5, SHA256 and bcrypt have different downside and benefits each algorithm has.
# Being able to utilize salts has shown me a practical demonstration of how to deal with rainbow table attacks. Also working with peppers have shown me how having another security layer can result in stronger security. When experimenting with the TOTP script, I was able to understand how the integration of a time based token can be carried out as a method to act as a second factor. 
# Working practically with bcrypt demonstrated how a slow and adaptive hashing function can result in dictionary attacks being less effective. 
# Overall through this seminar, I was able to connect the concepts learned in the lecture with practical code which gave me an deeper understanding of how these concepts tie into how systems are protected in real applications.